# Graph cut method for bloodcells counting

A lot of thanks for Paweł :)

This method strongly relies on parameters, that accomodates capacities of edges accordingly to image's outlook.
Below code is tested out for bloodcells segmentation. 
IMPORTANT NOTE: Approach presented below needs a proper model change, because of its time consumption and segmentation effects.

In [1]:
from PIL import Image
import numpy as np
import networkx as nx
from matplotlib import pyplot as plt
from scipy.stats import norm
from networkx.algorithms.flow import edmonds_karp

Necessary functions for pipeline:

In [2]:
def get_distribution(image):
    im_data = np.array(image.getchannel('G'),dtype='int64')
    im_data = im_data.reshape(im_data.shape[0] * im_data.shape[1], 1)
    mu, std = norm.fit(im_data)
    return mu, std

def get_bitmap(R, threshold, dimx, dimy):
    bitmap = np.zeros((dimx, dimy))
    for i in range(dimx):
        for j in range(dimy):
            if R['s'][(i,j)]['flow'] > threshold:
                bitmap[i][j] = 1
    return bitmap

def create_graph(arr, mu, std, k, s):
    G = nx.DiGraph()
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            G.add_node((i,j))
    G.add_node('s')
    G.add_node('t')
    for y in range(arr.shape[1]):
        for x in range(arr.shape[0]):
            if x > 0:
                G.add_edge((x,y),(x-1,y),capacity=(k * np.exp((abs(arr[x,y] - arr[x-1,y])) ** 2 / s)))
                G.add_edge((x-1,y),(x,y),capacity=(k * np.exp((abs(arr[x,y] - arr[x-1,y])) ** 2 / s)))
            if y > 0:
                G.add_edge((x,y),(x,y-1),capacity=(k * np.exp((abs(arr[x,y] - arr[x,y-1])) ** 2 / s)))
                G.add_edge((x,y-1),(x,y),capacity=(k * np.exp((abs(arr[x,y] - arr[x,y-1])) ** 2 / s)))
            G.add_edge((x,y),'t',capacity=(1 - norm(mu, std).pdf(arr[x,y])))
            G.add_edge('s',(x,y),capacity=(norm(mu, std).pdf(arr[x,y])))
    return G

Pipeline:

In [3]:
def image_processing_pipeline(image):
    
    # image is an opened picture
    image_arr = np.array(image,dtype='int64')
    
    width, height, depth = image_arr.shape
    
    result_image = np.zeros((width, height, depth))
    
    # find a fitting distribution for background
    mu, std = get_distribution(image)
    
    w_coords = list(range(0, width, 32))
    h_coords = list(range(0, height, 32))
    
    if w_coords[-1] != width - 1:
        w_coords.append(width - 1)
    if h_coords[-1] != height - 1:
        h_coords.append(height - 1)

    for d in range(depth):
        # we split the image into 32x32 subimages
        for w in range(len(w_coords) - 1): # not considering last element
            for h in range(len(h_coords) - 1):
                arr = image_arr[w_coords[w]:w_coords[w+1], h_coords[h]:h_coords[h+1], d]
                
                G = create_graph(arr, mu, std, 0.0025, 10)
                
                R = edmonds_karp(G, 's', 't')
                
                print('Current flow:', R.graph['flow_value'], end=' ')
                
                # compute possible treshold for current digraph
                mean = R.graph['flow_value'] / len(G.nodes())
                
                dimx = w_coords[w+1] - w_coords[w]
                dimy = h_coords[h+1] - h_coords[h]
                bitmap = get_bitmap(R, mean, dimx, dimy)
                
                arr2 = arr * bitmap
                
                result_image[w_coords[w]:w_coords[w+1], h_coords[h]:h_coords[h+1], d] = arr2
                
                print('Slice {}:{},{}:{},{} done!'.format(w_coords[w], w_coords[w+1], h_coords[h], h_coords[h+1], d))

    return result_image